In [ ]:
import torch
from torch.utils.dataset import Dataset
from torchvision.datasets import ImageFolder
from torchvision import Transforms

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
os.listdir('/content/drive/MyDrive/Colab Notebooks/age_detection/data')

['age_detection.csv', 'test', 'train']

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/age_detection/data'

Link PDF: https://drive.google.com/drive/u/0/folders/1MZ9lhzGKABXLMTJzeIM2b-zi7YJnn6WR